In [1]:
from model.saprot.base import SaprotBaseModel
from transformers import EsmTokenizer

/home/sp2530/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = {
    "task": "base",
    "config_path": "/home/sp2530/Desktop/SaProt/model/saprot/SaProt_650M_AF2/", # Note this is the directory path of SaProt, not the ".pt" file
    "load_pretrained": True,
}

model = SaprotBaseModel(**config)
tokenizer = EsmTokenizer.from_pretrained(config["config_path"])

device = "cuda"
model.to(device)

seq = "M#EvVpQpL#VyQdYaKv" # Here "#" represents lower plDDT regions (plddt < 70)
tokens = tokenizer.tokenize(seq)
print(tokens)

inputs = tokenizer(seq, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

embeddings = model.get_hidden_states(inputs, reduction="mean")
print(embeddings[0].shape)

Some weights of EsmForMaskedLM were not initialized from the model checkpoint at /home/sp2530/Desktop/SaProt/model/saprot/SaProt_650M_AF2/ and are newly initialized: ['esm.contact_head.regression.weight', 'esm.embeddings.position_embeddings.weight', 'esm.contact_head.regression.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['M#', 'Ev', 'Vp', 'Qp', 'L#', 'Vy', 'Qd', 'Ya', 'Kv']
torch.Size([1280])


In [3]:
embeddings

[tensor([-0.0243,  0.1525,  0.1106,  ..., -0.0122, -0.0740, -0.0749],
        device='cuda:0', grad_fn=<MeanBackward1>)]

# Convert protein structure into structure-aware sequence

In [4]:
from utils.foldseek_util import get_struc_seq
pdb_path = "example/8ac8.cif"

# Extract the "A" chain from the pdb file and encode it into a struc_seq
# pLDDT masks low-confidence regions if "plddt_mask" is True. Please set it to True when
# use AF2 structures for best performance.
parsed_seqs = get_struc_seq("foldseek/foldseek", pdb_path, ["A"], plddt_mask=False)["A"]
seq, foldseek_seq, combined_seq = parsed_seqs

print(f"seq: {seq}")
print(f"foldseek_seq: {foldseek_seq}")
print(f"combined_seq: {combined_seq}")

seq: NSPRIVQSNDLTEAAYSLSRDQKRMLYLFVDQIRKSDDGICEIHVAKYAEIFGLTSAEASKDIRQALKSFAGKEVVFYYESFPWFIKPAHSPSRGLYSVHINPYLIPFFIGLQNRFTQFRLSETKEITNPYAMRLYESLCQYRKPDGSGIVSLKIDWIIERYQLPQSYQRMPDFRRRFLQVCVNEINSRTPMRLSYIEKKKGRQTTHIVFSFRDITS
foldseek_seq: DFQKAKFAVQQLLKAWEAAPLLVLVVSVQLSCLAVVVQQKDKDALVVSCVVLVHDSVVSVVSVVVRLVVLPPTWIAGVHDIDRQFPDRWDAPDVRMIITGGDSVRSVRRPPDDDNMFIDGPQQCSQQRGVLLVSVVRVQSSAADPQQWGKDKDWLVSSCRRNVDDPQCRPPVSVCPVPVPVSLVRSCVRGQKNKDWDWDDDPPDTTIIMMTIGGVVD
combined_seq: NdSfPqRkIaVkQfSaNvDqLqTlElAkAaYwSeLaSaRpDlQlKvRlMvLvYsLvFqVlDsQcIlRaKvSvDvDqGqIkCdEkIdHaVlAvKvYsAcEvIvFlGvLhTdSsAvEvAsSvKvDsIvRvQvArLlKvSvFlApGpKtEwViVaFgYvYhEdSiFdPrWqFfIpKdPrAwHdSaPpSdRvGrLmYiSiVtHgIgNdPsYvLrIsPvFrFrIpGpLdQdNdRnFmTfQiFdRgLpSqEqTcKsEqIqTrNgPvYlAlMvRsLvYvErSvLqCsQsYaRaKdPpDqGqSwGgIkVdSkLdKwIlDvWsIsIcErRrYnQvLdPdQpSqYcQrRpMpPvDsFvRcRpRvFpLvQpVvCsVlNvErIsNcSvRrTgPqMkRnLkSdYwIdEwKdKdKdGpRpQdTtTtHiIiVmFmStFiRgDgIvTvSd
